In [14]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker datasets --quiet

In [15]:
import sagemaker, boto3,json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session("us-west-2").region_name
sess = sagemaker.Session()

print(aws_role)
print(aws_region)
print(sess)

arn:aws:iam::466280163741:role/service-role/AmazonSageMaker-ExecutionRole-20241004T175497
us-west-2


In [19]:
(model_id, model_version,) = ("meta-textgeneration-llama-2-7b","2.*",)

In [20]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version, instance_type="ml.g5.2xlarge")
predictor = model.deploy()
# predictor = model.deploy(accept_eula=True)

For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.
Using vulnerable JumpStart model 'meta-textgeneration-llama-2-7b' and version '2.1.8'.
Using model 'meta-textgeneration-llama-2-7b' with wildcard version identifier '2.*'. You can pin to version '2.1.8' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


----------------------!

In [21]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

In [22]:
payload = {
    "inputs": "Genomic characterization is essential for",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

Genomic characterization is essential for
>  the establishment of a reference genome that can be used for comparative genomic analyses.
We sequenced the genomes of 1055 individuals from 111 populations of the world (Fig. 1, Table 1). The 111 populations were chosen to represent all




In [23]:
# Delete the SageMaker endpoint and the attached resources
predictor.delete_model()
predictor.delete_endpoint()